In [1]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import pprint
import os

In [ ]:
# --- 1. Configuración de la Conexión ---
# Por defecto, se conecta a localhost en el puerto 27017.
CONNECTION_STRING = os.getenv('COSMOS_DB_CONNECTION_STRING')

# --- 2. Conexión al Servidor de CosmosDB ---
try:
    # Crear una instancia del cliente de CosmosDB
    client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)
    # Forzar una conexión para verificar que el servidor está disponible
    client.admin.command('ping')
    print("✅ Conexión a CosmosDB exitosa.")
except ConnectionFailure as e:
    print(f"❌ No se pudo conectar a CosmosDB: {e}")
    # Detener la ejecución si no se puede conectar
    client = None

# --- 3. Listar Bases de Datos y Colecciones ---
if client:
    print("\n--- Listado de Bases de Datos y Colecciones ---")
    
    try:
        # Obtener la lista de nombres de todas las bases de datos
        lista_db = client.list_database_names()

        # Filtrar bases de datos internas del sistema para un listado más limpio
        db_a_excluir = {'admin', 'config', 'local'}
        bases_de_datos_usuario = [db_nombre for db_nombre in lista_db if db_nombre not in db_a_excluir]

        if not bases_de_datos_usuario:
            print("No se encontraron bases de datos de usuario.")
        else:
            # Iterar sobre cada base de datos para obtener sus colecciones
            for db_nombre in bases_de_datos_usuario:
                print(f"\n📁 Base de Datos: {db_nombre}")
                
                # Seleccionar la base de datos
                db = client[db_nombre]
                
                # Obtener la lista de colecciones en la base de datos actual
                lista_colecciones = db.list_collection_names()
                
                if not lista_colecciones:
                    print("   (No contiene colecciones)")
                else:
                    for col_nombre in lista_colecciones:
                        # Contar documentos en cada colección para dar más contexto
                        num_documentos = db[col_nombre].count_documents({})
                        print(f"   - 📄 Colección: {col_nombre} ({num_documentos} documentos)")

    except Exception as e:
        print(f"Ocurrió un error al listar las bases de datos: {e}")
        
    finally:
        # --- 4. Cerrar la Conexión ---
        # Es una buena práctica cerrar siempre la conexión al finalizar
        client.close()
        print("\n✅ Conexión cerrada.")

/tmp/ipykernel_27869/202745178.py:8: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(CONNECTION_STRING, serverSelectionTimeoutMS=5000)


✅ Conexión a MongoDB exitosa.

--- Listado de Bases de Datos y Colecciones ---

📁 Base de Datos: ajover-chatbot-db
   - 📄 Colección: session-collection (4 documentos)
   - 📄 Colección: users-collection (1 documentos)

✅ Conexión cerrada.


In [6]:
def revisar_coleccion(
    coleccion_a_revisar: str,
    db_a_revisar: str = "ajover-chatbot-db",
    limite_documentos: int = 10,
    connection_string: str = CONNECTION_STRING
):
    """
    Se conecta a una base de datos de CosmosDB, revisa una colección específica
    y muestra un número limitado de sus documentos.

    Args:
        coleccion_a_revisar (str): El nombre de la colección a inspeccionar.
        db_a_revisar (str): El nombre de la base de datos.
        limite_documentos (int): El número máximo de documentos a mostrar.
        connection_string (str): La cadena de conexión a CosmosDB.
    """
    client = None
    try:
        # --- 1. Conexión al Servidor de CosmosDB ---
        client = MongoClient(connection_string, serverSelectionTimeoutMS=5000)
        client.admin.command('ping')
        print("✅ Conexión a CosmosDB exitosa.")
    except ConnectionFailure as e:
        print(f"❌ No se pudo conectar a CosmosDB: {e}")
        return  # Terminar la ejecución si no hay conexión

    try:
        # --- 2. Revisar los Documentos de la Colección ---
        if db_a_revisar not in client.list_database_names():
            print(f"❌ La base de datos '{db_a_revisar}' no existe.")
            return

        db = client[db_a_revisar]
        
        if coleccion_a_revisar not in db.list_collection_names():
            print(f"❌ La colección '{coleccion_a_revisar}' no existe en la base de datos '{db_a_revisar}'.")
            return

        coleccion = db[coleccion_a_revisar]
        
        print(f"\n--- Mostrando los primeros {limite_documentos} documentos de la colección: '{coleccion_a_revisar}' ---")
        
        documentos = coleccion.find().limit(limite_documentos)
        total_documentos = coleccion.count_documents({})
        print(f"(Colección con un total de {total_documentos} documentos)\n")

        docs_encontrados = 0
        for doc in documentos:
            pprint.pprint(doc)
            print("-" * 40)  # Separador
            docs_encontrados += 1
        
        if docs_encontrados == 0:
            print("La colección está vacía o no se encontraron documentos.")

    except Exception as e:
        print(f"Ocurrió un error al revisar los documentos: {e}")
        
    finally:
        # --- 3. Cerrar la Conexión ---
        if client:
            client.close()
            print("\n✅ Conexión cerrada.")

In [7]:
# --- Ejemplo de Uso ---
if __name__ == "__main__":
    # Revisar la colección 'user-collection' (usando los parámetros por defecto)
    print("Revisando la colección 'users-collection'...")
    revisar_coleccion("users-collection")
    print("\n" + "="*50 + "\n")

    # Ejemplo cambiando también el límite de documentos
    revisar_coleccion("session-collection")

Revisando la colección 'users-collection'...


/tmp/ipykernel_27869/74447474.py:20: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(connection_string, serverSelectionTimeoutMS=5000)


✅ Conexión a MongoDB exitosa.

--- Mostrando los primeros 10 documentos de la colección: 'users-collection' ---
(Colección con un total de 1 documentos)

{'_id': 'wZLfmmXXOoUbwcz-j0AHKYqAmgKv706UnM0doUhR',
 'email': 'test-c2104592-246c-4bc8-8e9c-11278e1a5792@example.com',
 'hashed_password': '$2b$12$ZCSByEKz.hWx0hyk8/gNK.6ax3fzpnzy1M4ny5FmxVkjuWBMy/Cma',
 'is_active': True,
 'user_id': 'wZLfmmXXOoUbwcz-j0AHKYqAmgKv706UnM0doUhR',
 'user_type': 'external'}
----------------------------------------

✅ Conexión cerrada.


✅ Conexión a MongoDB exitosa.

--- Mostrando los primeros 10 documentos de la colección: 'session-collection' ---
(Colección con un total de 4 documentos)

{'_id': 'sess_23bdc0c17670a708c4784a893f96707d',
 'chat_history': [],
 'created_at': '2025-06-26T16:14:49.670344-05:00',
 'session_id': 'sess_23bdc0c17670a708c4784a893f96707d',
 'status': 'active',
 'updated_at': '2025-06-26T16:14:49.670348-05:00',
 'user_id': '1279cf50-a82d-4890-ad1d-e0a574d411e9',
 'user_type': 'exter